In [0]:
%python
spark.conf.set(
    "fs.azure.account.key.storagell1.dfs.core.windows.net",
    "BIYE4A5YIPi0IZeOV6Wpmoe/VLhvI3jGQALvWIIY+NqJBw/aD2b3w+8ZYwa79uYXAU5UjByNSexx+AStnAiU9w=="
)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7021941727743998>, line 1
----> 1 spark.conf.set(
      2     "fs.azure.account.key.storagell1.dfs.core.windows.net",
      3     "BIYE4A5YIPi0IZeOV6Wpmoe/VLhvI3jGQALvWIIY+NqJBw/aD2b3w+8ZYwa79uYXAU5UjByNSexx+AStnAiU9w=="
      4 )

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/conf.py:46, in RuntimeConf.set(self, key, value)
     44 op_set = proto.ConfigRequest.Set(pairs=[proto.KeyValue(key=key, value=value)])
     45 operation = proto.ConfigRequest.Operation(set=op_set)
---> 46 result = self._client.config(operation)
     47 for warn in result.warnings:
     48     warnings.warn(warn)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1858, in SparkConnectClient.config(self, operation)
   1856     raise SparkConnectException("Invalid state during retry excep

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, DateType, BooleanType
from pyspark.sql.functions import col, year

In [0]:
hourly_data_schema = StructType(fields =
[
    StructField("instant", IntegerType(), False),
    StructField("dteday", DateType(), True),
    StructField("season", IntegerType(), True),
    StructField("yr", IntegerType(), True),
    StructField("mnth", IntegerType(), True),
    StructField("hr", IntegerType(), True),
    StructField("holiday", IntegerType(), True),
    StructField("weekday", IntegerType(), True),
    StructField("workingday", IntegerType(), True),
    StructField("weathersit", IntegerType(), True),
    StructField("temp", DoubleType(), True),
    StructField("atemp", DoubleType(), True),
    StructField("hum", DoubleType(), True),
    StructField("windspeed", DoubleType(), True),
    StructField("casual", IntegerType(), True),
    StructField("registered", IntegerType(), True),
    StructField("cnt", IntegerType(), True)
])

In [0]:
hourly_df = spark.read.option("header", True).schema(hourly_data_schema).csv("abfss://container1@storagell1.dfs.core.windows.net/bikesharingdataset/hour.csv")

In [0]:
hourly_df.display()

In [0]:
hourly_df.printSchema()

In [0]:
hourly_renamed_df = hourly_df.withColumnRenamed("instant", "id") \
                            .withColumnRenamed("dteday", "date_") \
                            .withColumnRenamed("mnth", "month_") \
                            .withColumnRenamed("hr", "hour_") \
                            .withColumnRenamed("weekday", "week_day") \
                            .withColumnRenamed("workingday", "working_day") \
                            .withColumnRenamed("weathersit", "weather_id") \
                            .withColumnRenamed("temp", "temperature") \
                            .withColumnRenamed("atemp", "feel_like_temperature") \
                            .withColumnRenamed("hum", "humidity") \
                            .withColumnRenamed("windspeed", "wind_speed") \
                            .withColumnRenamed("casual", "casual_riders") \
                            .withColumnRenamed("registered", "registeres_riders") \
                            .withColumnRenamed("cnt", "total_riders")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-7021941727744012>, line 1
----> 1 hourly_renamed_df = hourly_df.withColumnRenamed("instant", "id") \
      2                             .withColumnRenamed("dteday", "date_") \
      3                             .withColumnRenamed("mnth", "month_") \
      4                             .withColumnRenamed("hr", "hour_") \
      5                             .withColumnRenamed("weekday", "week_day") \
      6                             .withColumnRenamed("workingday", "working_day") \
      7                             .withColumnRenamed("weathersit", "weather_id") \
      8                             .withColumnRenamed("temp", "temperature") \
      9                             .withColumnRenamed("atemp", "feel_like_temperature") \
     10                             .withColumnRenamed("hum", "humidity") \
     11     

In [0]:
hourly_dropped_df = hourly_renamed_df.drop(col('yr'))

In [0]:
hourly_final_df = hourly_dropped_df.withColumn("year", year(col("date_")))

In [0]:
hourly_final_df.write.mode("overwrite").partitionBy("year").format("delta").saveAsTable("bike_sharing_silver_mayur.hourly_full_data_mayur")

In [0]:
%sql 
describe formatted bike_sharing_silver_mayur.hourly_full_data_mayur